# Setting Node Capacities

Capacities of a FABRIC node are basic characteristics of the virtual machine including number of compute core, amount of memory, and amount of local disk.  This notebook will demonstrate the options for setting these node capciites.

## Configure the Environment

In [ ]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV4'] = '192.168.11.226'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV6'] = '2600:2701:5000:a902::c'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']=<INSERT_YOUR_FABRIC_USERNAME>
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/.ssh/id_rsa_fabric'

# Set the keypair FABRIC will install in your slice. Only set the passphrase if your 
# private key is requires one.
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE'] = ''

## Setup the Experiment

#### Import FABRIC API

In [ ]:
from fabrictestbed_extensions.fablib.fablib import fablib

## Configure Slice Parameters



In [ ]:
slice_name='MySlice'
node_name='node1'
site='MAX'
image='default_centos_8'

## Setting Capacities

We are going to be creating slices that contain one node each.

We need to specify the resources (number of cores, amount of ram and amount of disk space) that we want to allocate to our node.

We can do that in two ways:
 - Using Capacities()
 - Using capacity hints.

## Example 1: Exact Capacities

Let's create our first slice that contains one node. We will use `Capacities()` to specify the resources that we want to allocate.

The line `cap.set_fields(core=2, ram=8, disk=10)` specifies that we want to reserve a node with 2 cores, 8GB of RAM and 10GB of disk.

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Add node
    node = slice.add_node(name=node_name, site=site)
    node.set_capacities(cores=2, ram=8, disk=10)
    node.set_image(image)

    #Submit Slice Request
    slice.submit(wait_progress=True)
except Exception as e:
    print(f"Slice Fail: {e}")

Get the slice and topology

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"   Name              : {node.get_name()}")
        print(f"   Cores             : {node.get_cores()}")
        print(f"   RAM               : {node.get_ram()}")
        print(f"   Disk              : {node.get_disk()}")
        print()    
except Exception as e:
    print(f"Fail: {e}")

It says that our node has 2 cores, 8GB of RAM and 10GB of disk space, which is what we requested.

Now let's delete the slice.

In [ ]:
return_status, result = slice_manager.delete(slice_object=slice)
print("Response Status {}".format(return_status))

## Example 2: Rounded Capacities

Now let's try something else. Let's try to request 2 cores, 8GB of RAM, and 50GB of disk space.

Again, we are going to use `cap.set_fields(core=2, ram=8, disk=50)`.

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Add node
    node = slice.add_node(name=node_name, site=site)
    node.set_capacities(cores=2, ram=8, disk=50)
    node.set_image(image, username)

    #Submit Slice Request
    slice.submit(wait_progress=True)
except Exception as e:
    print(f"Slice Fail: {e}")

Get the slice and topology

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"   Name              : {node.get_name()}")
        print(f"   Cores             : {node.get_cores()}")
        print(f"   RAM               : {node.get_ram()}")
        print(f"   Disk              : {node.get_disk()}")
        print()    
except Exception as e:
    print(f"Fail: {e}")

We can see that we were allocated 2 cores, 8GB of ram, but 100GB of disk space instead of 50GB.

The reason for this is that we have discrete "capacity hints". The node can only be an instance of one of those capacity hints.

See the very last cell in this notebook for the complete list of available capacity hints.

This is an exerpt of the available capacity hints. Full list available [here](https://github.com/fabric-testbed/InformationModel/blob/master/fim/slivers/data/instance_sizes.json).

    "fabric.c16.m64.d10": {"core":16, "ram":64, "disk": 10},
    "fabric.c32.m128.d10": {"core":32, "ram":128, "disk": 10},
    "fabric.c1.m4.d100": {"core":1, "ram":4, "disk": 100},
    "fabric.c2.m8.d100": {"core":2, "ram":8, "disk": 100},
    "fabric.c4.m16.d100": {"core":4, "ram":16, "disk": 100},

We can see that the disk space can only be 10GB or 100GB. So when we requested 50GB, it was rounded up to 100GB.

### Now let's delete the slice.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Error: {e}")

## Example 3: Instance Type

Finally, we can directly set the resources that we need choosing an instance tupe. _Please see the very last cell in this notebook for the complete list of available instance types._

We can set the needed resources like so:

`node.set_instance_type('fabric.c2.m8.d10')`.

This would reserve a node with 2 processor cores, 8GB of memory and 10GB of disk space.
 - The number next to the `c` is the number of cores.
 - The number next to the `m` is the amount of memory in GB.
 - The number next to the `d` is the amount of disk space in GB.

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Add node
    node = slice.add_node(name=node_name, site=site)
    node.set_instance_type('fabric.c2.m8.d10')
    node.set_image(image, username)

    #Submit Slice Request
    slice.submit(wait_progress=True)
except Exception as e:
    print(f"Slice Fail: {e}")
    traceback.print_exc()
    


Get the slice and topology

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"Slice: {slice.get_name()}")
except Exception as e:
    print(f"Get Slices Fail: {e}")
    traceback.print_exc()

Print the allocated capacities

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"   Name              : {node.get_name()}")
        print(f"   Cores             : {node.get_cores()}")
        print(f"   RAM               : {node.get_ram()}")
        print(f"   Disk              : {node.get_disk()}")
        print()    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

We can see that we got the resources that we requested.

Now let's delete the slice.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

## Capacity hints (and their descriptions) below. 

Full list available [here](https://github.com/fabric-testbed/InformationModel/blob/master/fim/slivers/data/instance_sizes.json).

    {
      "fabric.c1.m4.d10": {"core":1, "ram":4, "disk": 10},
      "fabric.c2.m8.d10": {"core":2, "ram":8, "disk": 10},
      "fabric.c4.m16.d10": {"core":4, "ram":16, "disk": 10},
      "fabric.c8.m32.d10": {"core":8, "ram":32, "disk": 10},
      "fabric.c16.m64.d10": {"core":16, "ram":64, "disk": 10},
      "fabric.c32.m128.d10": {"core":32, "ram":128, "disk": 10},
      "fabric.c1.m4.d100": {"core":1, "ram":4, "disk": 100},
      "fabric.c2.m8.d100": {"core":2, "ram":8, "disk": 100},
      "fabric.c4.m16.d100": {"core":4, "ram":16, "disk": 100},
      "fabric.c8.m32.d100": {"core":8, "ram":32, "disk": 100},
      "fabric.c16.m64.d100": {"core":16, "ram":64, "disk": 100},
      "fabric.c32.m128.d100": {"core":32, "ram":128, "disk": 100},
      "fabric.c1.m4.d500": {"core":1, "ram":4, "disk": 500},
      "fabric.c2.m8.d500": {"core":2, "ram":8, "disk": 500},
      "fabric.c4.m16.d500": {"core":4, "ram":16, "disk": 500},
      "fabric.c8.m32.d500": {"core":8, "ram":32, "disk": 500},
      "fabric.c16.m64.d500": {"core":16, "ram":64, "disk": 500},
      "fabric.c32.m128.d500": {"core":32, "ram":128, "disk": 500},
      "fabric.c1.m4.d2000": {"core":1, "ram":4, "disk": 2000},
      "fabric.c2.m8.d2000": {"core":2, "ram":8, "disk": 2000},
      "fabric.c4.m16.d2000": {"core":4, "ram":16, "disk": 2000},
      "fabric.c8.m32.d2000": {"core":8, "ram":32, "disk": 2000},
      "fabric.c16.m64.d2000": {"core":16, "ram":64, "disk": 2000},
      "fabric.c32.m128.d2000": {"core":32, "ram":128, "disk": 2000},
      "fabric.c64.m384.d4000": {"core":64, "ram":384, "disk": 4000}
    }